In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: udacity-project
Azure region: eastus
Subscription id: e8f628b3-bb5b-4edf-947a-8637ca6ea7c2
Resource group: aml-quickstarts-112782


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "MLCPU"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

Found existing cpucluster


In [18]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os
from azureml.core import ScriptRunConfig
from azureml.train.estimator import Estimator

# Specify parameter sampler
ps = RandomParameterSampling(
    {
       '--optimizer': choice('Adam', 'SGD')
    }
)

# Specify a Policy
#policy = BanditPolicy(evaluation_interval=2, slack_factor=0.2)
policy = MedianStoppingPolicy()
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py


script_params = {
       '--kernel': 'linear',
       '--penalty': 1.0,
   }

project_folder = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlcpu/code'

# estimator = Estimator(source_directory=project_folder,
#                        #script_params=script_params,
#                        compute_target=cpu_cluster,
#                        entry_script='train.py',
#                        pip_packages=['joblib==0.13.2']
#                       )
src = ScriptRunConfig(source_directory=project_folder,
                       compute_target=cpu_cluster,
                       script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# src = ScriptRunConfig(project_folder)

hdc = HyperDriveConfig(estimator=estimator, 
                          hyperparameter_sampling=ps, 
                          policy=policy, 
                          primary_metric_name='accuracy', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=1,
                          max_concurrent_runs=1)



In [19]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hd_run = exp.submit(hdc)
RunDetails(hd_run).show()

### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.
#best_run, fitted_model = hd_run.get_output()
best_run = hd_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print(best_run)
joblib.dump(best_run, 'model.joblib')

### YOUR CODE HERE ###

In [20]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.data.datapath import DataPath
from azureml.core import Dataset, Datastore
# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = Dataset.Tabular.from_delimited_files(path=path)
#print(ds.to_pandas_dataframe())
### YOUR CODE HERE ###

In [21]:
from train import clean_data
# import train
# Use the clean_data function to clean your data.

x, y = clean_data(ds)

In [22]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    iterations=5,
    task='classification',
    primary_metric='AUC_weighted',
    X=x,
    y=y,
    n_cross_validations=5)

WARNING - The AutoMLConfig parameters, X and y, will soon be deprecated. Please refer to our documentation for the latest interface: https://aka.ms/AutoMLConfig


In [23]:
# Submit your automl run
#from azureml.core.experiment import Experiment

#ws = Workspace.from_config()

# Choose a name for the experiment and specify the project folder.
#experiment_name = 'automl-classification'

#experiment = Experiment(ws, experiment_name)

run = exp.submit(automl_config, show_output=True)

### YOUR CODE HERE ###

Running on local machine
Parent Run ID: AutoML_5fae3360-1fb7-445a-adde-8531f2363ee2

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely p

In [24]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [26]:
# Retrieve and save your best automl model.
import joblib
best_run, fitted_model = run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, 'model_automl.joblib')

### YOUR CODE HERE ###


Run(Experiment: udacity-project,
Id: AutoML_5fae3360-1fb7-445a-adde-8531f2363ee2_3,
Type: None,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               max_depth=3,
                                                                                               min_child_weight=1,
                                                      

['model_automl.joblib']

In [11]:
pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlcpu/code'